In [42]:
import requests
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import os


In [23]:
os.chdir(r"/Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/tdubolyou.github.io/crime")

# Dictionary of dataset names and their respective URLs
datasets = {
    "mci": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Major_Crime_Indicators_Open_Data/FeatureServer/0/query",
    "hom": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Homicides_Open_Data_ASR_RC_TBL_002/FeatureServer/0/query",
    "gun": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Shooting_and_Firearm_Discharges_Open_Data/FeatureServer/0/query",
    "bike": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Bicycle_Thefts_Open_Data/FeatureServer/0/query",
    "car":"https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Theft_From_Motor_Vehicle_Open_Data/FeatureServer/0/query",
    "nbr":"https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Neighbourhood_Crime_Rates_Open_Data/FeatureServer/0/query"
    # Add more datasets as needed
}

# Initialize a dictionary to store the GeoDataFrames
gdfs = {}

for dataset_name, url in datasets.items():
    all_features = []
    offset = 0
    while True:
        # Define request parameters
        params = {
            "where": "1=1",
            "outFields": "*",
            "outSR": "4326",
            "f": "geojson",
            "resultRecordCount": 2000,
            "resultOffset": offset
        }
        
        # Make the request
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            features = data.get("features", [])
            if not features:
                break  # Exit loop if no more features to fetch
            all_features.extend(features)
            offset += len(features)
        else:
            print(f"Failed to fetch data for {dataset_name}. Status code: {response.status_code}")
            break

    # Convert all_features to a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(all_features)
    
    # Save the GeoDataFrame to a GeoJSON file
    filename = f"data/{dataset_name}.geojson"
    gdf.to_file(filename, driver='GeoJSON')

    gdfs[f'gdf_{dataset_name}'] = gdf
    print(filename+" downloaded")

# At this point, gdfs contains a GeoDataFrame for each dataset, accessible via its name, e.g., gdfs['gdf_dataset1']


data/mci.geojson downloaded
data/hom.geojson downloaded
data/gun.geojson downloaded
data/bike.geojson downloaded
data/car.geojson downloaded
data/nbr.geojson downloaded


In [6]:
os.chdir('/Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/tdubolyou.github.io/crime')
dir = os.getcwd()
print(dir)


/Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/tdubolyou.github.io/crime


In [78]:
gdf_mci = gpd.read_file(dir+"/data/"+'mci.geojson')
gdf_hom = gpd.read_file(dir+"/data/"+'hom.geojson')
gdf_gun = gpd.read_file(dir+"/data/"+'gun.geojson')
gdf_bike = gpd.read_file(dir+"/data/"+'bike.geojson')
gdf_car = gpd.read_file(dir+"/data/"+'car.geojson')
gdf_nbr = gpd.read_file(dir+"/data/"+'nbr.geojson')

In [79]:
#Filter down Redundant GDFS
gdf_gun = gdf_gun[gdf_gun['DEATH'] == 0] #Remove shooting deaths as they are redundant with homicides
gdf_car = gdf_car[gdf_car['MCI_CATEGORY'] == 'NonMCI']

In [80]:
##Add a crime field 
# Example for the Homicide GDF
gdf_hom['CRIME'] = 'Homicide'

# Example for the Non-Fatal Shootings GDF, assuming 'INJURIES' indicates non-fatal shootings
gdf_gun['CRIME'] = 'Shooting (Non-Fatal)'

# Example for the Bike Theft GDF
gdf_bike['CRIME'] = 'Bike Theft'

# For the MCI GDF, assuming 'MCI_CATEGORY' is present and you want to exclude 'NonMCI' values
gdf_mci['CRIME'] = gdf_mci['MCI_CATEGORY']

gdf_car['CRIME'] = 'Theft From Motor Vehicle'

# If there's a specific condition for other datasets, follow a similar pattern:
# gdf_x['CRIME'] = [appropriate logic here]


In [84]:
data = [gdf_mci, gdf_hom, gdf_gun, gdf_bike, gdf_car]
for gdf in data:

    print(gdf.columns)
    print(len(gdf))
    print(gdf['CRIME'].value_counts())

Index(['OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE', 'REPORT_YEAR',
       'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW', 'REPORT_HOUR',
       'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR',
       'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE', 'UCR_EXT',
       'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84', 'geometry', 'CRIME'],
      dtype='object')
372899
CRIME
Assault            197906
Break and Enter     70148
Auto Theft          58441
Robbery             33921
Theft Over          12483
Name: count, dtype: int64
Index(['OBJECTID', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR', 'OCC_MONTH',
       'OCC_DAY', 'OCC_DOW', 'OCC_DOY', 'DIVISION', 'HOMICIDE_TYPE',
       'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140',
       'LONG_WGS84', 'LAT_WGS84', 'geometry', 'CRIME'],
      dtype='object')
1396
CRIME
Homicide    1396
Name: count, dtyp

In [85]:
# Convert columns Index to a list and print
print(list(gdf_nbr.columns))


['OBJECTID_1', 'AREA_NAME', 'HOOD_ID', 'ASSAULT_2014', 'ASSAULT_2015', 'ASSAULT_2016', 'ASSAULT_2017', 'ASSAULT_2018', 'ASSAULT_2019', 'ASSAULT_2020', 'ASSAULT_2021', 'ASSAULT_2022', 'ASSAULT_2023', 'ASSAULT_RATE_2014', 'ASSAULT_RATE_2015', 'ASSAULT_RATE_2016', 'ASSAULT_RATE_2017', 'ASSAULT_RATE_2018', 'ASSAULT_RATE_2019', 'ASSAULT_RATE_2020', 'ASSAULT_RATE_2021', 'ASSAULT_RATE_2022', 'ASSAULT_RATE_2023', 'AUTOTHEFT_2014', 'AUTOTHEFT_2015', 'AUTOTHEFT_2016', 'AUTOTHEFT_2017', 'AUTOTHEFT_2018', 'AUTOTHEFT_2019', 'AUTOTHEFT_2020', 'AUTOTHEFT_2021', 'AUTOTHEFT_2022', 'AUTOTHEFT_2023', 'AUTOTHEFT_RATE_2014', 'AUTOTHEFT_RATE_2015', 'AUTOTHEFT_RATE_2016', 'AUTOTHEFT_RATE_2017', 'AUTOTHEFT_RATE_2018', 'AUTOTHEFT_RATE_2019', 'AUTOTHEFT_RATE_2020', 'AUTOTHEFT_RATE_2021', 'AUTOTHEFT_RATE_2022', 'AUTOTHEFT_RATE_2023', 'BIKETHEFT_2014', 'BIKETHEFT_2015', 'BIKETHEFT_2016', 'BIKETHEFT_2017', 'BIKETHEFT_2018', 'BIKETHEFT_2019', 'BIKETHEFT_2020', 'BIKETHEFT_2021', 'BIKETHEFT_2022', 'BIKETHEFT_2023', '

In [86]:
# Common columns identified (as an example)
common_columns = ['EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR', 'OCC_DOW','DIVISION','HOOD_158','NEIGHBOURHOOD_158', 'CRIME','geometry']

# Additional necessary columns identified (example based on your provided columns)
additional_columns = ['OCC_HOUR','HOMICIDE_TYPE', 'INJURIES', 'BIKE_MAKE', 'BIKE_COST', 'STATUS', 'OFFENCE', 'PRIMARY_OFFENCE','LOCATION_TYPE', 'PREMISES_TYPE',]

In [88]:
for gdf in data:
    for col in common_columns:
        print(col in gdf_hom.columns)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [89]:
# Standardize GeoDataFrames
gdfs = [gdf_mci, gdf_hom, gdf_gun, gdf_bike, gdf_car]
for gdf in gdfs:
    for col in additional_columns:
        if col not in gdf.columns:
            gdf[col] = None  # Fill missing additional columns with None or an appropriate value

for gdf in gdfs:

    print(gdf.columns)
    print(len(gdf))



Index(['OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE', 'REPORT_YEAR',
       'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW', 'REPORT_HOUR',
       'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR',
       'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE', 'UCR_EXT',
       'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84', 'geometry', 'CRIME',
       'HOMICIDE_TYPE', 'INJURIES', 'BIKE_MAKE', 'BIKE_COST', 'STATUS',
       'PRIMARY_OFFENCE'],
      dtype='object')
372899
Index(['OBJECTID', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR', 'OCC_MONTH',
       'OCC_DAY', 'OCC_DOW', 'OCC_DOY', 'DIVISION', 'HOMICIDE_TYPE',
       'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140',
       'LONG_WGS84', 'LAT_WGS84', 'geometry', 'CRIME', 'OCC_HOUR', 'INJURIES',
       'BIKE_MAKE', 'BIKE_COST', 'STATUS', 'OFFENCE', 'PRIMARY_OFFENCE',
       'LOCATION_TYPE', 'PREMISES_T

In [90]:
# Select columns for each GDF before concatenating
gdfs = [gdf[common_columns + additional_columns] for gdf in gdfs if set(common_columns).issubset(gdf.columns)]

# Merge standardized GeoDataFrames
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Save merged GeoDataFrame to a new GeoJSON file
merged_gdf.to_file(dir+"/data/crime.geojson", driver='GeoJSON')

In [93]:
print(merged_gdf.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 501964 entries, 0 to 501963
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   EVENT_UNIQUE_ID    501964 non-null  object  
 1   OCC_DATE           501964 non-null  int64   
 2   OCC_YEAR           501834 non-null  float64 
 3   OCC_DOW            501834 non-null  object  
 4   DIVISION           501964 non-null  object  
 5   HOOD_158           501964 non-null  object  
 6   NEIGHBOURHOOD_158  501964 non-null  object  
 7   CRIME              501964 non-null  object  
 8   geometry           501964 non-null  geometry
 9   OCC_HOUR           500568 non-null  object  
 10  HOMICIDE_TYPE      1396 non-null    object  
 11  INJURIES           5364 non-null    object  
 12  BIKE_MAKE          34985 non-null   object  
 13  BIKE_COST          32607 non-null   float64 
 14  STATUS             34985 non-null   object  
 15  OFFENCE            460219 

In [95]:
print(merged_gdf['CRIME'].value_counts())

CRIME
Assault                     197906
Theft From Motor Vehicle     87320
Break and Enter              70148
Auto Theft                   58441
Bike Theft                   34985
Robbery                      33921
Theft Over                   12483
Shooting (Non-Fatal)          5364
Homicide                      1396
Name: count, dtype: int64


In [103]:
####Add HOOD_ID to Merged_gdf and create individual Neighbourhood point files

# Ensure both GeoDataFrames use the same CRS
merged_gdf = merged_gdf.to_crs(gdf_nbr.crs)
print(merged_gdf.crs)

# Perform a spatial join
merged_gdf = gpd.sjoin(merged_gdf, gdf_nbr[['HOOD_ID', 'geometry']], how='left', op='within')

print(merged_gdf.columns)

##Create individual Neighbourhood geojsons
for neighborhood_id in joined_gdf['HOOD_ID'].unique():
    # Filter the GeoDataFrame for the current neighborhood ID
    gdf_subset = merged_gdf[merged_gdf['HOOD_ID'] == neighborhood_id]
    
    # Define the filename using the neighborhood ID
    filename = f'/data/crime_{neighborhood_id}.geojson'
    
    # Save the subset GeoDataFrame to a GeoJSON file
    gdf_subset.to_file(dir+filename, driver='GeoJSON')
    
    print(f'Saved: {filename}')

EPSG:4326


/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [115]:
###Create Annual Population Fields for gdf_nbr
rate_base = 100000  # Standard rate base for crime rates

# Loop through each year to calculate population based on ASSAULT and ASSAULT_RATE
for year in range(2014, 2023):
    total_incidents_column = f'ASSAULT_{year}'
    rate_column = f'ASSAULT_RATE_{year}'
    population_column = f'POPULATION_{year}'

    # Calculate population for each year
    # Ensure to handle division by zero or NaN values in rates
    gdf_nbr[population_column] = (gdf_nbr[total_incidents_column] / (gdf_nbr[rate_column] / rate_base)).fillna(0).replace([np.inf, -np.inf], 0)

print(gdf_nbr.columns)

Index(['OBJECTID_1', 'AREA_NAME', 'HOOD_ID', 'ASSAULT_2014', 'ASSAULT_2015',
       'ASSAULT_2016', 'ASSAULT_2017', 'ASSAULT_2018', 'ASSAULT_2019',
       'ASSAULT_2020',
       ...
       'geometry', 'POPULATION_2014', 'POPULATION_2015', 'POPULATION_2016',
       'POPULATION_2017', 'POPULATION_2018', 'POPULATION_2019',
       'POPULATION_2020', 'POPULATION_2021', 'POPULATION_2022'],
      dtype='object', length=196)


In [116]:
print(gdf_nbr.head(10))

   OBJECTID_1                  AREA_NAME HOOD_ID  ASSAULT_2014  ASSAULT_2015   
0           1  South Eglinton-Davisville     174            63            61  \
1           2              North Toronto     173            45            52   
2           3         Dovercourt Village     172            56            57   
3           4   Junction-Wallace Emerson     171           154           157   
4           5         Yonge-Bay Corridor     170           394           524   
5           6             Bay-Cloverhill     169           104           100   
6           7        Bendale-Glen Andrew     156           137           148   
7           8                  Downsview     155           114           127   
8           9   Oakdale-Beverley Heights     154           221           232   
9          10                   Avondale     153            38            40   

   ASSAULT_2016  ASSAULT_2017  ASSAULT_2018  ASSAULT_2019  ASSAULT_2020   
0            70            82            85 

In [105]:
##Create a new gdf_nbr with the geometry and summaries of the incidents by CRIME and OCC_YEAR
merged_gdf['OCC_YEAR'] = merged_gdf['OCC_YEAR'].astype('Int64')
merged_gdf['HOOD_ID'] = merged_gdf['HOOD_ID'].astype(str)

# Filter out rows where OCC_YEAR is NaN in addition to being less than 2014
filtered_gdf = merged_gdf[pd.notna(merged_gdf['OCC_YEAR']) & (merged_gdf['OCC_YEAR'] >= 2014)]


# Group by HOOD_ID, OCC_YEAR, and CRIME, then sum the Incidents
filtered_gdf['Incidents'] = 1
grouped = filtered_gdf.groupby(['HOOD_ID', 'OCC_YEAR', 'CRIME'])['Incidents'].sum().reset_index()

# Initial pivot operation
pivot_table = grouped.pivot_table(index='HOOD_ID', 
                                  columns=['OCC_YEAR', 'CRIME'], 
                                  values='Incidents', 
                                  fill_value=0)

# Create new column names by concatenating CRIME and OCC_YEAR
pivot_table.columns = [crime + '_' + str(year) for year, crime in pivot_table.columns]

# Reset the index to turn HOOD_ID back into a column
pivot_table.reset_index(inplace=True)

# Select only 'HOOD_ID' and 'geometry' from gdf_nbr for the merge
gdf_nbr_selected = gdf_nbr[['HOOD_ID', 'geometry','POPULATION_2014', 'POPULATION_2015', 'POPULATION_2016',
       'POPULATION_2017', 'POPULATION_2018', 'POPULATION_2019',
       'POPULATION_2020', 'POPULATION_2021', 'POPULATION_2022']]


# Ensure data types match for the merging key
gdf_nbr_selected['HOOD_ID'] = gdf_nbr_selected['HOOD_ID'].astype(str)
pivot_table['HOOD_ID'] = pivot_table['HOOD_ID'].astype(str)

# Perform the merge
gdf_nbr_new = gdf_nbr_selected.merge(pivot_table, on='HOOD_ID', how='left')

print(gdf_nbr_new.columns)

/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
### CREATE A CHLOROPLETH MAP OF NBR

import folium

# Convert the GeoDataFrame 'nbr' to GeoJSON format
nbr_geojson = gdf_nbr.to_json()

# Create a map centered around Toronto
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=11.5)

# Add the choropleth layer to visualize 'Assault_rate_2023'
folium.Choropleth(
    geo_data=nbr_geojson,  # Use the GeoJSON representation of the 'nbr' GeoDataFrame
    name="choropleth",
    data=gdf_nbr,  # The DataFrame providing the data (GeoDataFrame in this case)
    columns=['HOOD_ID', 'ASSAULT_RATE_2023'],  # Assuming there's an 'id' column in 'nbr' for the neighborhood; adjust if it's different
    key_on='feature.properties.HOOD_ID',  # Path in GeoJSON features to the neighborhood identifier; adjust based on your actual GeoJSON structure
    fill_color='YlGnBu',  # Using a different color scheme for variety
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Assault Rate in Toronto by Neighborhood (2023)'
).add_to(map_toronto)

# Display or save the map
# Uncomment the next line to display in a Jupyter notebook
# map_toronto

# Save to an HTML file
#map_toronto.save('toronto_assault_rate_choropleth_map.html')

map_toronto
